<a href="https://colab.research.google.com/github/BRama10/Bayesian-ML-Notebooks/blob/main/Bayesian_Opt_vs_Random_vs_Grid_on_large_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tuning with relatively larger datasets

In [ ]:
#Display runtime for individual code cells
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
time: 3.38 ms (started: 2022-07-29 12:56:37 +00:00)


In [ ]:
#Import scikit-optimize and necessary  methods
!pip install scikit-optimize
import skopt
from sklearn.datasets import make_classification
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, StratifiedShuffleSplit
from scipy.stats import randint, uniform
import pandas as pd
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer


print('skopt %s' % skopt.__version__)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 100 kB 5.9 MB/s 
skopt 0.9.0
time: 12.8 s (started: 2022-07-29 12:56:39 +00:00)


In [ ]:
#Create a classification dataset with 120 features, 50,000 rows
X2, y2 = make_classification(n_samples=50000, n_features=120, n_redundant=50, n_informative=70,random_state=1)

time: 803 ms (started: 2022-07-29 12:56:52 +00:00)


In [ ]:
X2_df = pd.DataFrame(X2)
X2_df

,0,1,2,3,4,5,6,7,8,9,...,110,111,112,113,114,115,116,117,118,119
0,-1.674055,4.228774,1.838958,-8.752498,1.777974,2.818265,-25.320768,-3.635645,-2.777827,-4.847355,...,-11.853227,5.340038,3.240460,8.230972,-4.415916,7.282014,-5.105974,-8.657978,-3.894829,1.046275
1,-0.318122,-0.471437,-2.167085,11.348309,-5.809443,-9.145150,-42.725349,5.741191,4.602851,-2.193046,...,-8.895566,2.814138,-18.950767,3.659573,-1.596285,-3.387228,0.624439,1.915039,-20.795351,-4.728479
2,1.852562,4.193240,-1.989371,-5.724476,5.059104,-6.389138,-28.853856,3.089784,10.030269,4.940933,...,36.607270,-1.313438,-4.022826,-0.904014,-5.454733,-4.950902,9.889486,7.790807,32.189028,-3.376335
3,-3.333856,-4.667652,1.485149,6.136802,8.109175,-26.473137,-16.438581,-2.163745,-6.652577,1.420240,...,35.939946,-5.134005,52.530624,-2.800791,1.111053,-2.267078,10.286922,-6.780645,-1.841098,1.645096
4,2.451969,-4.685324,-2.759618,-0.147699,-3.833817,49.465705,2.974990,-0.576148,-1.407105,2.623111,...,22.385070,-2.084986,22.917386,-8.745481,1.528107,-2.664407,-6.838039,-4.402623,10.822547,5.657787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,5.497450,-0.763741,5.753113,7.961158,2.365366,13.730964,-1.285960,7.478088,3.049141,-6.962272,...,4.055790,3.266253,34.875661,-4.644539,-1.781533,14.678586,6.272757,-8.695046,34.535113,-17.864443
49996,-5.140493,-4.557967,-2.510070,6.362068,7.870979,41.114260,-24.158964,-11.639759,3.557874,2.860537,...,-16.125424,-9.202038,47.247681,-2.135918,-0.638346,-9.515656,-6.088022,1.450604,-6.710290,4.996595
49997,-14.086636,-9.633415,2.180021,4.030280,-2.245727,15.723606,17.787863,6.622888,-18.085021,0.260303,...,5.349431,-4.517144,10.731272,1.904824,-6.758169,6.950214,-0.080462,1.282629,-14.387706,-0.946491
49998,3.148983,-6.316484,0.952515,-4.664803,15.719626,-0.044620,26.917086,-1.203313,-9.129199,0.000610,...,-4.292043,0.380421,-19.721241,0.920813,5.153343,1.047771,-2.163012,3.062611,24.922729,17.274148


time: 50.8 ms (started: 2022-07-29 12:56:54 +00:00)


In [ ]:
y2_df = pd.DataFrame(y2)
y2_df

,0
0,1
1,0
2,1
3,1
4,0
...,...
49995,0
49996,0
49997,0
49998,1


time: 63.2 ms (started: 2022-07-29 12:56:57 +00:00)


In [ ]:
clf = XGBClassifier(random_state=42, n_jobs=-1)

time: 1.79 ms (started: 2022-07-29 01:16:15 +00:00)


In [ ]:
#Grid Search
params = {
    'learning_rate': [.1]
    , 'subsample': [0.2, 0.3 ,0.4, 0.5]
    , 'n_estimators': [25, 50]
    , 'min_child_weight': [25]
    , 'reg_alpha': [0.3, 0.4, 0.5]
    , 'reg_lambda': [0.1, 0.2, 0.3, 0.4, 0.5]
    , 'colsample_bytree': [0.66]
    , 'max_depth': [5]
}


grid = GridSearchCV(clf, param_grid=params, cv=3, n_jobs=-1, scoring='accuracy', return_train_score=True)
grid.fit(X2, y2)
print(grid.best_score_)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9208999823363667
time: 37min 22s (started: 2022-07-28 14:44:52 +00:00)


In [ ]:
#Randomized Search
params = {
    'learning_rate': uniform(.05, .1)
    , 'subsample': uniform(.2, .3)
    , 'n_estimators': randint(20, 70)
    , 'min_child_weight': randint(20, 40)
    , 'reg_alpha': uniform(0, .7)
    , 'reg_lambda': uniform(0, .7)
    , 'colsample_bytree': uniform(.1, .7)
    , 'max_depth': randint(2, 6)
}
random = RandomizedSearchCV(clf, param_distributions=params, n_iter=20, cv=StratifiedShuffleSplit(n_splits=1, test_size=.25, random_state=1), n_jobs=-1, scoring='accuracy', return_train_score=True)
random.fit(X2, y2)
print(random.best_score_)




0.90728
time: 1min 45s (started: 2022-07-28 15:25:50 +00:00)


In [ ]:
#Bayes Search
params = {
    'learning_rate': Real(.05, .1+.05)
    , 'subsample': Real(.2, .5)
    , 'n_estimators': Integer(20, 70)
    , 'min_child_weight': Integer(20, 40)
    , 'reg_alpha': Real(0, 0+.7)
    , 'reg_lambda': Real(0, 0+.7)
    , 'colsample_bytree': Real(.1, .1+.7)
    , 'max_depth': Integer(2, 6)
}
bayes = BayesSearchCV(clf, params, n_iter=20, scoring='accuracy', n_jobs=-1, cv=StratifiedShuffleSplit(n_splits=1, test_size=.25, random_state=1))
bayes.fit(X2, y2)
print(bayes.best_score_)

0.95352
time: 3min 52s (started: 2022-07-28 15:29:18 +00:00)


#Diabetes sklearn - Hypertuning with multiple models for regressors

In [ ]:
#Import data
import pandas as pd

db = pd.read_csv('https://raw.githubusercontent.com/coreyjwade/bca_datasets/master/diabetes.csv', nrows = 200)

db

X1 = db.iloc[:,:-1]
y1=db.iloc[:,-1]
db

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
195,5,158,84,41,210,39.4,0.395,29,1
196,1,105,58,0,0,24.3,0.187,21,0
197,3,107,62,13,48,22.9,0.678,23,1
198,4,109,64,44,99,34.8,0.905,26,1


time: 291 ms (started: 2022-07-29 01:30:13 +00:00)


In [ ]:
#Import data, split into train & test

from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
from skopt.plots import plot_objective, plot_histogram
from skopt.utils import use_named_args
from skopt.plots import plot_evaluations
from skopt import gp_minimize
from skopt.plots import plot_convergence

from sklearn.datasets import load_diabetes
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator

import numpy as np
import pandas as pd

diabetes = load_diabetes()
db = pd.DataFrame(data=np.c_[diabetes['data'], diabetes['target']], 
                           columns=diabetes['feature_names'] + ['target'])
db

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641,135.0
...,...,...,...,...,...,...,...,...,...,...,...
437,0.041708,0.050680,0.019662,0.059744,-0.005697,-0.002566,-0.028674,-0.002592,0.031193,0.007207,178.0
438,-0.005515,0.050680,-0.015906,-0.067642,0.049341,0.079165,-0.028674,0.034309,-0.018118,0.044485,104.0
439,0.041708,0.050680,-0.015906,0.017282,-0.037344,-0.013840,-0.024993,-0.011080,-0.046879,0.015491,132.0
440,-0.045472,-0.044642,0.039062,0.001215,0.016318,0.015283,-0.028674,0.026560,0.044528,-0.025930,220.0


time: 48.8 ms (started: 2022-07-29 12:57:21 +00:00)


In [ ]:
#Split into train & test datasets
X_train, X_test, y_train, y_test = train_test_split(db.drop(columns="target"), db.target, test_size=0.2, random_state=21)

time: 13.9 ms (started: 2022-07-29 12:57:23 +00:00)


###Outdated

In [ ]:
#Create placeholder estimator
class DummyEstimator(BaseEstimator):
  def fit(self): pass
  def score(self): pass
        
#Create pipeline
pipe = Pipeline([('model', DummyEstimator())]) 

#Create search space
search_space = [{'model': [AdaBoostRegressor()], # Actual Estimator
                  'model__learning_rate': [0.005, 0.007, 0.009, 0.01, 0.03, 0.05, 0.09,0.1,0.2,0.3,0.4,0.5,0.6,0.9],
                  'model__n_estimators': [1,10,20,30,40,50,60,70,80,110,120,130,200,400,450,500,670,980],
                  'model__loss':['linear', 'square', 'exponential']},
                    
                    {'model': [GradientBoostingRegressor()],
                     'model__learning_rate': [0.005, 0.007, 0.009, 0.01, 0.03, 0.05, 0.09,0.1,0.2,0.3,0.4,0.5,0.6,0.9],
                      'model__n_estimators': [1,10,20,30,40,50,60,70,80,110,120,130,200,400,450,500,670,980],
                      'model__loss': ['squared_error', 'absolute_error', 'quantile']}]
#Call Grid-Search
optim = GridSearchCV(
    pipe,
    search_space,
    cv=5
)
#Fit data
optim.fit(X_train, y_train)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:290: FutureWarning: The loss 'ls' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:290: FutureWarning: The loss 'ls' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:290: FutureWarning: The loss 'ls' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:290: FutureWarning: The loss 'ls' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:290: FutureWarning: The loss '

GridSearchCV(cv=5, estimator=Pipeline(steps=[('model', DummyEstimator())]),
             param_grid=[{'model': [AdaBoostRegressor(learning_rate=0.5,
                                                      loss='exponential',
                                                      n_estimators=30)],
                          'model__learning_rate': [0.005, 0.007, 0.009, 0.01,
                                                   0.03, 0.05, 0.09, 0.1, 0.2,
                                                   0.3, 0.4, 0.5, 0.6, 0.9],
                          'model__loss': ['linear', 'square', 'exponential'],
                          'model__n_estimators': [1, 10, 20, 30, 40, 50, 60, 70,
                                                  80, 110, 120, 130, 200, 400,
                                                  450, 500, 670, 980]},
                         {'model': [GradientBoostingRegressor()],
                          'model__learning_rate': [0.005, 0.007, 0.009, 0.01,
                

time: 51min 11s (started: 2022-07-29 02:01:13 +00:00)


###Grid & Bayes Regressor tuning

In [ ]:
class DummyEstimator(BaseEstimator):
  def fit(self): pass
  def score(self): pass
        

pipe_grid = Pipeline([('model', DummyEstimator())]) 


search_space = [{'model': [AdaBoostRegressor()], 
                  'model__learning_rate': np.array(np.linspace(0.005, 0.9, 15)),
                  'model__n_estimators': np.array(np.linspace(1,1000,15,dtype=int)),
                  'model__loss':['linear', 'square', 'exponential']},
                    
                    {'model': [GradientBoostingRegressor()],
                     'model__learning_rate': np.array(np.linspace(0.005, 0.9, 15)),
                      'model__n_estimators': np.array(np.linspace(1,1000,15,dtype=int)),
                      'model__loss': ['squared_error', 'absolute_error', 'quantile']}]
optim_grid = GridSearchCV(
    pipe_grid,
    search_space,
    cv=5
)

optim_grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Pipeline(steps=[('model', DummyEstimator())]),
             param_grid=[{'model': [AdaBoostRegressor(learning_rate=0.13285714285714287,
                                                      loss='square',
                                                      n_estimators=72)],
                          'model__learning_rate': array([0.005     , 0.06892857, 0.13285714, 0.19678571, 0.26071429,
       0.32464286, 0.38857143, 0.4525    , 0.51642857, 0.58035714,
       0.64428571, 0.70821429, 0.77214286, 0.83607...
                         {'model': [GradientBoostingRegressor()],
                          'model__learning_rate': array([0.005     , 0.06892857, 0.13285714, 0.19678571, 0.26071429,
       0.32464286, 0.38857143, 0.4525    , 0.51642857, 0.58035714,
       0.64428571, 0.70821429, 0.77214286, 0.83607143, 0.9       ]),
                          'model__loss': ['squared_error', 'absolute_error',
                                          'quantile'],
    

time: 1h 35min 8s (started: 2022-07-29 14:03:57 +00:00)


In [ ]:
class DummyEstimator(BaseEstimator):
  def fit(self): pass
  def score(self): pass
        
# Create a pipeline
pipe_bayes = Pipeline([('model', DummyEstimator())]) # Placeholder Estimator

ada_bayes = {
    'model': [AdaBoostRegressor()],
    'model__learning_rate': Real(0.005, 0.9, prior="log-uniform"),
    'model__n_estimators': Integer(1, 1000),
    'model__loss': Categorical(['linear', 'square', 'exponential'])
}

gb_bayes = {
    'model': [GradientBoostingRegressor()],
    'model__learning_rate': Real(0.005, 0.9, prior="log-uniform"),
    'model__n_estimators': Integer(1, 1000), 
    'model__loss': Categorical(['squared_error', 'absolute_error', 'quantile'])
}

optim_bayes = BayesSearchCV(
    pipe_bayes,
    [(ada_bayes, 100), (gb_bayes, 100)],
    cv=5
)

optim_bayes.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.7/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.7/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


BayesSearchCV(cv=5, estimator=Pipeline(steps=[('model', DummyEstimator())]),
              search_spaces=[({'model': [AdaBoostRegressor(learning_rate=0.1880354440925412,
                                                           loss='square',
                                                           n_estimators=66)],
                               'model__learning_rate': Real(low=0.005, high=0.9, prior='log-uniform', transform='normalize'),
                               'model__loss': Categorical(categories=('linear', 'square', 'exponential'), prior=...
                               'model__n_estimators': Integer(low=1, high=1000, prior='uniform', transform='normalize')},
                              100),
                             ({'model': [GradientBoostingRegressor()],
                               'model__learning_rate': Real(low=0.005, high=0.9, prior='log-uniform', transform='normalize'),
                               'model__loss': Categorical(categories=('squared_er

time: 26min 40s (started: 2022-07-29 13:27:16 +00:00)


In [ ]:
print("Bayes Best Score is " + str(optim_bayes.best_score_))
print("Grid Best Score is " + str(optim_grid.best_score_))


Bayes Best Score is 0.4667420753605949
Grid Best Score is 0.46685197442410437
time: 2.1 ms (started: 2022-07-29 16:41:11 +00:00)


In [ ]:
print("Bayes Best Params are " + str(optim_bayes.best_params_) +'\n\n\n\n\n\n')
print("Grid Best Params are " + str(optim_grid.best_params_))

Bayes Best Params are OrderedDict([('model', AdaBoostRegressor(learning_rate=0.1880354440925412, loss='square',
                  n_estimators=66)), ('model__learning_rate', 0.1880354440925412), ('model__loss', 'square'), ('model__n_estimators', 66)])






Grid Best Params are {'model': AdaBoostRegressor(learning_rate=0.13285714285714287, loss='square',
                  n_estimators=72), 'model__learning_rate': 0.13285714285714287, 'model__loss': 'square', 'model__n_estimators': 72}
time: 6.98 ms (started: 2022-07-29 16:50:06 +00:00)
